In [ ]:
# Hi Yuxi
# this code will replace the existing row counting code in your deployment
# You can remove the initial part of the code as you have already implemented it in your ai deployment. 
import os
import shutil
import random
import glob
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# this section reads all the txt files and append it at the end of data frame (df)
# You have written a similar code ( make sure the order of variables is same ({'class' : [],'xc' : [],'yc' : [],'x' : [],'y' : [],'# frame':[]}))


tic = time.time()
txt_file = []
df = pd.DataFrame({'class' : [],'xc' : [],'yc' : [],'x' : [],'y' : [],'# frame':[]})

my_files = glob.glob('*.txt')
for name in my_files:
    txt_file.append(int(name.split('_')[-1].split('.')[0])) #change 3 if file name is different standard file name "HPWB6364_Trim_1.txt"
    df1 = pd.read_csv(name,sep=' ',header=None)
    df1.columns = ['class','xc','yc','x','y']
    df1['# frame'] = int(name.split('_')[-1].split('.')[0])-1
    #print(df1)
    df = df.append(df1, ignore_index=True)
    df = df.sort_values(by=['# frame'],ignore_index=True)
print(df)

      class        xc        yc         x         y  # frame
0       0.0  0.479167  0.227083  0.015625  0.019444      0.0
1       0.0  0.342969  0.280556  0.016146  0.019444      0.0
2       0.0  0.264583  0.631250  0.016667  0.020833      0.0
3       0.0  0.236458  0.586806  0.016667  0.022222      0.0
4       0.0  0.168750  0.571181  0.016667  0.021528      0.0
...     ...       ...       ...       ...       ...      ...
7166    0.0  0.294531  0.199653  0.018229  0.022917    236.0
7167    0.0  0.664844  0.848264  0.020313  0.024306    236.0
7168    0.0  0.660937  0.598611  0.016667  0.022222    236.0
7169    0.0  0.357812  0.455903  0.019792  0.024306    236.0
7170    0.0  0.363802  0.726389  0.018229  0.023611    236.0

[7171 rows x 6 columns]


In [ ]:
# this section defines a new dataframe (d) which takes all the values from df and sort the rows based on frame number.
# This code also assigns default id as -1
# this id is used to match identification accross different frames

d = df
d.reset_index(drop="index",inplace=True)
d=d.sort_values(by = ['# frame'], ascending = [True]) 
d['id'] = -1
d

,class,xc,yc,x,y,# frame,id
0,0.0,0.479167,0.227083,0.015625,0.019444,0.0,-1
21,0.0,0.429948,0.275000,0.016146,0.020833,0.0,-1
22,0.0,0.604167,0.414236,0.017708,0.021528,0.0,-1
23,0.0,0.261719,0.843056,0.014063,0.016667,0.0,-1
24,0.0,0.289583,0.495833,0.018750,0.022222,0.0,-1
...,...,...,...,...,...,...,...
423,0.0,0.575000,0.339931,0.016667,0.021528,11.0,-1
424,0.0,0.223698,0.388542,0.016146,0.020139,11.0,-1
425,0.0,0.271615,0.138542,0.017188,0.020139,11.0,-1
427,0.0,0.510156,0.707639,0.016146,0.020833,11.0,-1


In [ ]:
#this is the main body of code which uses the dataframe d and manipulates it to match objects across frame
# the input to this code is dataframe "d" created in previous section.
#with loop vectorized
import warnings
warnings.filterwarnings("ignore")
tic = time.time()
d['id']=-1 # default id
n = 15  # the tails would be searched in next 5 frames to avoid loosing particle
id = 0 #first id # increases with every new trail
theta = 15 # +/- angle acceptable deflection of next position 
fp = d.iloc[0] # first/lead particle assigned to varialble
fid = d.index[0] # The index of firsst particle in database variable 'd'

while (True):    #breaks when all the identified particles are given id (id != -1) 
    if(fp['id']==-1): # allots new id if id is -1 (unassigned)
        fp['id']=id
        id = id+1;
    temp = d[(d['# frame']>fp['# frame']) & (d['# frame']<=fp['# frame']+n) & (d['id']==-1)] #temp stores data of next n frames where trail is searched
    temp = temp[temp['xc']>fp['xc']]
    temp['theta'] = 200 # to store the angle theta between 2 positions wrt to horizontal axis
    temp['dis'] = 500   # distance between 2 positions 
    temp['theta'] = np.abs((180/np.pi)*np.arctan((temp['yc']-fp[['yc']].to_numpy())/(temp['xc']-fp[['xc']].to_numpy()))) #calculating theta
    temp['dis'] = np.sqrt((temp['yc']-fp[['yc']].to_numpy())**2+(temp['xc']-fp[['xc']].to_numpy())**2) # calculating distance   
    temp = temp[temp['theta']<theta] #filter to remove positions not within angular deviation  
    temp=temp.sort_values(by = ['# frame','theta','dis'], ascending = [True,True,True]) #sorting to select postion nearest frame
    d.at[fid,'id']=fp['id'] #updating the data base with id of lead particle 
    if(len(temp>0)): # checks if there are any trails ahead
        d.loc[temp.index[0],'id']=fp['id'] #updates data base with next position
        temp.iloc[0,-3]=fp['id'] #storing id to transfer it to next prediction
        fp = temp.iloc[0]
        fid = temp.index[0] 
        #print(temp)
    else:
        if(len(d[d['id']==-1])>0):
            fp = d[d['id']==-1].iloc[0]
            fid = d[d['id']==-1].index[0]
            #print('else')
        elif(len(d[d['id']==-1])==0):
            break
    #print(temp)
toc = time.time()
print("Time taken to process files=",-1*(tic-toc))
len(d['id'].unique())

Time taken to process files= 3.388153076171875


86